<a href="https://colab.research.google.com/github/ma850419/Various_scripts/blob/main/from_gee_to_colab_archeology.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()
ee.Initialize(project='velvety-ring-328419')

In [ ]:
# Install geemap if not already installed
!pip install geemap



In [ ]:
!pip install /content/ee-packages-py-main/

In [ ]:

import geemap


# Initialize Earth Engine (assuming Authenticate() has been run previously)
ee.Initialize(project='velvety-ring-328419')

# Define region of interest (South America)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'South America'))
SA = mask.geometry()

# Load MODIS NDVI data and mosaic
modis_col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI').filterBounds(SA).filterDate('2023-01-01', '2023-12-31')
# Create a mosaic of the MODIS collection for visualization
modis_mosaic = modis_col.mosaic()

# Load Sentinel-2 (Optical Data) and mosaic
# Filter by bounds before filtering by date to be more efficient
sentinel2_col = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA).filterDate('2023-01-01', '2023-12-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) # Add cloud filter
# Create a mosaic of the Sentinel-2 collection
sentinel2_mosaic = sentinel2_col.mosaic()


# Load Sentinel-1 (SAR Data) and mosaic
# Filter by bounds before filtering by date to be more efficient
# Changed the collection ID from 'COPERNICUS/S1' to 'COPERNICUS/S1_GRD'
sentinel1_col = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA).filterDate('2023-01-01', '2023-12-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW')) # Filter for a specific mode
# Create a mosaic of the Sentinel-1 collection
sentinel1_mosaic = sentinel1_col.mosaic()


# Visualization parameters
modis_vis = {
    'min': 0,
    'max': 8000,
    'palette': ['red', 'yellow', 'green']  # Single-band, palette allowed
}
sentinel2_vis = {
    'bands': ['B4', 'B3', 'B2'],  # True color visualization
    'min': 0,
    'max': 3000
}
sentinel1_vis = {
    'min': -25,
    'max': 0
}

# Create map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Add the mosaic images to the map
m.addLayer(modis_mosaic, modis_vis, "MODIS NDVI Mosaic")
m.addLayer(sentinel2_mosaic, sentinel2_vis, "Sentinel-2 Mosaic (True Color)")
m.addLayer(sentinel1_mosaic, sentinel1_vis, "Sentinel-1 Mosaic (VV)")


# Add region mask
m.addLayer(mask, {'color': 'black'}, "South America Mask")

# Display the map
m

In [ ]:
#print(modis_col.size().getInfo())  # Should return >0 if images exist
print(sentinel2_col.size().getInfo())
#print(sentinel1_col.size().getInfo())


In [ ]:
import geemap
import ee

# Initialize Earth Engine
ee.Initialize(project='velvety-ring-328419')

# Define region of interest (South America)
mask = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017').filter(ee.Filter.eq('wld_rgn', 'South America'))
SA = mask.geometry()

# Function to group images by month and stitch them
def stitch_images_by_month(image_collection, year, month):
    filtered_col = image_collection.filter(ee.Filter.calendarRange(year, year, 'year')) \
                                   .filter(ee.Filter.calendarRange(month, month, 'month'))
    return filtered_col.mosaic().set('year', year, 'month', month)

# Load MODIS NDVI data and process by month
modis_col = ee.ImageCollection('MODIS/006/MOD13A2').select('NDVI').filterBounds(SA).filterDate('2023-04-01', '2023-08-31')
modis_stitched = stitch_images_by_month(modis_col, 2023, 6)  # Example for June

# Load Sentinel-2 data and process by month
sentinel2_col = ee.ImageCollection("COPERNICUS/S2").filterBounds(SA).filterDate('2023-04-01', '2023-08-31').filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
sentinel2_stitched = stitch_images_by_month(sentinel2_col, 2023, 6)

# Load Sentinel-1 data and process by month
sentinel1_col = ee.ImageCollection("COPERNICUS/S1_GRD").filterBounds(SA).filterDate('2023-04-01', '2023-08-31').filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')).filter(ee.Filter.eq('instrumentMode', 'IW'))
sentinel1_stitched = stitch_images_by_month(sentinel1_col, 2023, 6)

# Visualization parameters
modis_vis = {
    'min': 0,
    'max': 8000,
    'palette': ['red', 'yellow', 'green']
}
sentinel2_vis = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000
}
sentinel1_vis = {
    'min': -25,
    'max': 0
}

# Create map
m = geemap.Map(center=[-14.2, -51.9], zoom=4)

# Add stitched images to the map
m.addLayer(modis_stitched, modis_vis, "MODIS NDVI Stitched by Month")
m.addLayer(sentinel2_stitched, sentinel2_vis, "Sentinel-2 Stitched by Month")
m.addLayer(sentinel1_stitched, sentinel1_vis, "Sentinel-1 Stitched by Month")

# Add region mask
m.addLayer(mask, {'color': 'black'}, "South America Mask")

# Display the map
m


In [ ]:
#text = require('users/gena/packages:text');
#style = require('users/gena/packages:style');
#utils = require('users/gena/packages:utils');
min = -1;
max = 1;
textProperties = {
  'fontSize': 16,
  'textColor': 'red',
  'outlineColor': 'black',
  'outlineWidth': 0.1,
  'outlineOpacity': 0.6
};
labels = ee.List.sequence(min, max);
gradientBar = style.GradientBar.draw(geometry3, {
  'min': min, 'max': max, 'palette': visParams.palette, 'labels': labels,
  'format': '%.0f', 'text': textProperties});